# Toronto Dwellings Analysis Dashboard

In [1]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

C:\Users\Angelo\anaconda3\envs\pyvizenv\lib\site-packages\xarray\core\merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly") 

In [3]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(map_box_api)

## Import Data

In [4]:
# Import the CSVs to Pandas DataFrames
file_path = Path("toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path_2 = Path("toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

## Panel Visualizations

### Global available data

In [5]:
# Calculate the sum number of dwelling types units per year 
num_dwelling_per_year = to_data.groupby(to_data.index).sum()
num_dwelling_per_year = num_dwelling_per_year.drop(['average_house_value', 'shelter_costs_owned', 'shelter_costs_rented'], axis =1)

In [6]:
# Calculate the average monthly shelter costs for owned and rented dwellings
monthly_shelter_cost = to_data.groupby(to_data.index).mean()
monthly_shelter_cost= monthly_shelter_cost.drop(['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house','average_house_value'], axis =1)

In [7]:
# Calculate Average House values
average_house_value = to_data[["average_house_value"]].groupby("year").mean()

In [8]:
# Create a new DataFrame with the mean house values by neighbourhood per year
avg_house_value_by_neighbourhood = to_data
avg_house_value_by_neighbourhood = avg_house_value_by_neighbourhood.drop(['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house','shelter_costs_owned','shelter_costs_rented'], axis =1) 

In [9]:
# Fetch the data of all dwelling types per year
dwelling_types_per_year = to_data.reset_index()

In [10]:
# Getting the data from the top 10 expensive neighbourhoods
df_top_exp = to_data.groupby('neighbourhood').sum()
df_top_avg= df_top_exp['average_house_value'].sort_values(ascending=False).nlargest(10)

In [11]:
# Calculate the averagevalues for each neighborhood
average_neighbourhoods_value = to_data.groupby("neighbourhood").mean().reset_index()

In [12]:
# Join the average values with the neighbourhood locations
joined_avg_neighbourhood_value= pd.merge(df_neighbourhood_locations,average_neighbourhoods_value)

In [13]:
# Fetch the data from all expensive neighbourhoods per year
toronto_neighbourhoods = to_data.reset_index()

In [14]:
# Define Panel visualization functions
#create map box function
def neighbourhood_map():
    """Neighbourhood Map"""
    mapbox_scatterplot = px.scatter_mapbox(
        joined_avg_neighbourhood_value, 
        lat="lat", 
        lon="lon", 
        size="average_house_value", 
        color="average_house_value", 
        zoom=9,
        width=1500,
        color_continuous_scale=px.colors.cyclical.IceFire,
        title = "Average House Values in Toronto Neighbourhoods",
        hover_name = "neighbourhood",
        hover_data = ["single_detached_house", 
        "apartment_five_storeys_plus", 
        "movable_dwelling", 
        "semi_detached_house", 
        "row_house", 
        "duplex", 
        "apartment_five_storeys_less", 
        "other_house"])
    return mapbox_scatterplot

In [15]:
#create function for Bar charts
def create_bar_charts(data, title, xlabel, ylabel, color):
    """Create a barplot based in the data argument."""
    #creates bar chart based on agruments passed to the function
    fig = plt.figure()
    bar = data.plot.bar(color=color)
    bar.set_xlabel(xlabel)
    bar.set_ylabel(ylabel)
    bar.set_title(title)
    plt.show()
    plt.close(fig)
    
    return ax.figure

In [16]:
# create functions for line charts
def create_line_chart(data, title, xlabel, ylabel, color):
    """Create a line chart based in the data argument."""
    #creates line chart based on agruments passed to the function
    fig = plt.figure()
    line = data.plot.line(color= color)
    line.set_xlabel(xlabel)
    line.set_ylabel(ylabel)
    line.set_title(title) 
    plt.show()
    plt.close(fig)
    
    return ax.figure

In [17]:
def average_house_value():
    """Average house values per year."""
    ax=create_line_chart(avg_house_value,title="Average House Value in Toronto",xlabel="Year",ylabel="Avg House Value",color="blue")
    return ax

In [18]:
def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    ax=avg_house_value_by_neighbourhood.hvplot.line("year","average_house_value", groupby = "neighbourhood", xlabel="year",ylabel= "avg house value")
    return ax

In [19]:
def number_dwelling_types():
    """Number of dwelling types per year"""
    ax = dwelling_types_per_year_drop.hvplot.bar("year", groupby=["neighbourhood"], xlabel="year", ylabel="Dwelling Type Units", rot=90, height = 500)
    return ax
    
    # YOUR CODE HERE!

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    ax = px.bar(toronto_neighbourhoods,
                             title = "Average House Values in Toronto Neighbourhoods",
                             x = "neighbourhood",
                             y = "average_house_value",
                             facet_row = "year",
                             height = 1000,
                             width = 1400,
                             color= "average_house_value")



    
def top_most_expensive_neighbourhoods():

    ax = top10.head(10).hvplot.bar(x="neighbourhood", y="average_house_value", xlabel='Neighbourhood', ylabel='Avg. House Value', title="Top 10 Expensive Neighbourhoods in Toronto", height=700, width= 900, rot=90)

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    ax = expensive_to_neighbourhoods_subburstchart = px.sunburst(neighbourhood_sunburst_chart, 
                                                        path = ["year", "neighbourhood"], 
                                                        values = "average_house_value", 
                                                        hover_data = ["shelter_costs_owned", "shelter_costs_rented"], 
                                                        color = "shelter_costs_owned")
    return ax

# Panel Dashboard

In [24]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
title = pn.pane.Markdown("Real Estate Analysis of Toronto")

# Define a welcome text
# YOUR CODE HERE!

msg1="The dashboard presents a visual representation of historical house values, dwelling type per neighbourhood & dwelling cost in Toronto based on data from 2001 to 2016."

# Create a tab layout for the dashboard
# YOUR CODE HERE!

tabs = pn.Tabs(("Welcome",pn.Column(msg1, neighbourhood_map())),
    ("Yearly Market Analysis",
    pn.Column(pn.Row(create_bar_chart(num_dwelling_per_year.loc[2001], "Dwelling types in 2001", "2001", "Dwelling Types unit","red"),
    create_bar_chart(num_dwelling_per_year.loc[2006], "Dwelling types in 2006", "2006", "Dwelling Types unit","yellow")),
    pn.Row(create_bar_chart(num_dwelling_per_year.loc[2011], "Dwelling types in 2011", "2011", "Dwelling Types unit","blue"),
    create_bar_chart(num_dwelling_per_year.loc[2016], "Dwelling types in 2016", "2016", "Dwelling Types unit","purple")))),
    ("Shelter Cost vs House Value",pn.Column(pn.Row(create_line_chart(monthly_shelter_cost["shelter_costs_owned"], "Dwelling shelter cost onwed", "year", "Dwelling Types unit","blue")),
    create_line_chart(monthly_shelter_cost["shelter_costs_rented"], "Dwelling shelter cost rented", "year", "Dwelling Types unit","red"),
    create_line_chart(average_house_value, title="average_house_value", xlabel="", ylabel="Avg. House Value", color="blue")                                       
    )))                  

# Create the main dashboard
# YOUR CODE HERE!

dashboard = pn.Column(pn.Row(title), tabs, width = 1000, height = 800)

ValueError: Value of 'lat' is not the name of a column in 'data_frame'. Expected one of ['year', 'neighbourhood', 'single_detached_house', 'apartment_five_storeys_plus', 'movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house', 'average_house_value', 'shelter_costs_owned', 'shelter_costs_rented'] but received: lat